<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/Fine_Tune_Segformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Pillow
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -q transformers datasets evaluate segments-ai
!pip install rasterio scikit-image tensorflow keras gdown


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0

In [3]:
import os
import gdown
import zipfile
import rasterio
from skimage.transform import resize
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import random
from keras.utils import to_categorical
from skimage.util import random_noise

In [63]:
import os
from PIL import Image
import numpy as np

from torch.utils.data import Dataset


class SemanticSegmentationDataset(Dataset):
    """Image (semantic) segmentation dataset."""

    def __init__(self, root_dir, feature_extractor, train=True):
        """
        Args:
            root_dir (string): Root directory of the dataset containing the images + annotations.
            feature_extractor (SegFormerFeatureExtractor): feature extractor to prepare images + segmentation maps.
        """
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor

        self.img_dir = os.path.join(self.root_dir, "images")
        self.ann_dir = os.path.join(self.root_dir, "annotations")
        
        # read images
        image_file_names = []
        for root, dirs, files in os.walk(self.img_dir):
          image_file_names.extend(files)
        self.images = sorted(image_file_names)
        
        # read annotations
        annotation_file_names = []
        for root, dirs, files in os.walk(self.ann_dir):
          print(files)
          annotation_file_names.extend(files)
        self.annotations = sorted(annotation_file_names)

        assert len(self.images) == len(self.annotations), "There must be as many images as there are segmentation maps"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.img_dir, self.images[idx]))
        annotation = Image.open(os.path.join(self.ann_dir, self.annotations[idx]))

        # make 2D segmentation map (based on 3D one)
        # thanks a lot, Stackoverflow: https://stackoverflow.com/questions/61897492/finding-the-number-of-pixels-in-a-numpy-array-equal-to-a-given-color
        annotation = np.array(annotation)
        annotation_2d = np.zeros((annotation.shape[0], annotation.shape[1]), dtype=np.uint8) # height, width

        # Replace intensity values with corresponding labels
        for id, label in id2label.items():
            annotation[annotation == id] = label

        # randomly crop + pad both image and segmentation map to same size
        # feature extractor will also reduce labels!
        encoded_inputs = self.feature_extractor(image, Image.fromarray(annotation_2d), return_tensors="pt")

        for k,v in encoded_inputs.items():
          encoded_inputs[k].squeeze_() # remove batch dimension

        return encoded_inputs

In [4]:
# Download the training dataset
url = 'https://drive.google.com/uc?id=1f4eGmykyiczmNz2VPeNNmQ7aC7q8N_hD'
output = '/content/california_land_use.zip'
gdown.download(url, output, quiet=False)

# Extract the dataset
cwd = os.getcwd()
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(cwd+'/lulc')

Downloading...
From: https://drive.google.com/uc?id=1f4eGmykyiczmNz2VPeNNmQ7aC7q8N_hD
To: /content/california_land_use.zip
100%|██████████| 2.76G/2.76G [00:28<00:00, 95.8MB/s]


In [5]:
# Download the inference dataset
#https://drive.google.com/file/d/1mn00JDt51KlhyiVTfPJjRk7Ymzd_zB7n/view?usp=drive_link
url = 'https://drive.google.com/uc?id=1mn00JDt51KlhyiVTfPJjRk7Ymzd_zB7n'
output = '/content/california_land_use.zip'
gdown.download(url, output, quiet=False)

# Extract the dataset
cwd = os.getcwd()
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(cwd+'/sample_data/inference_data/')

Downloading...
From: https://drive.google.com/uc?id=1mn00JDt51KlhyiVTfPJjRk7Ymzd_zB7n
To: /content/california_land_use.zip
100%|██████████| 331M/331M [00:03<00:00, 97.6MB/s]


In [36]:
import rasterio
from PIL import Image
import numpy as np
import os

# # specify directories here
# input_dir = '/content/lulc/validation/rgbNIR'
# output_dir = '/content/lulc_png/validation/rgbNIR'

# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# for filename in os.listdir(input_dir):
#     if filename.endswith('.tif') or filename.endswith('.tiff'):
#         with rasterio.open(os.path.join(input_dir, filename)) as src:
#             array = src.read(out=np.zeros((src.count, src.height, src.width), dtype=np.uint8)) # using a uint8 array to hold the raster data
#             array = np.moveaxis(array, 0, -1)  # rasterio reads data in [bands, rows, cols] order, we transpose it to [rows, cols, bands] for PIL
#         img = Image.fromarray(array)
#         out = os.path.splitext(filename)[0] + '.png'
#         img.save(os.path.join(output_dir, out))
#         print('Converted: ', filename)

# print('All tif images have been converted to png.')


In [38]:
# # specify directories here
# input_dir = '/content/lulc/validation/labels'
# output_dir = '/content/lulc_png/validation/labels'

# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# for filename in os.listdir(input_dir):
#     if filename.endswith('.tif') or filename.endswith('.tiff'):
#         with rasterio.open(os.path.join(input_dir, filename)) as src:
#             array = src.read(1)  # This reads the first band into a 2D array
#             # If data is not uint8, scale or normalize data to 0-255 before conversion
#             if array.dtype != np.uint8:
#                 array = ((array - array.min()) * (1/(array.max() - array.min()) * 255)).astype('uint8')
#         img = Image.fromarray(array)
#         out = os.path.splitext(filename)[0] + '.png'
#         img.save(os.path.join(output_dir, out))
#         print('Converted: ', filename)

# print('All tif images have been converted to png.')

In [65]:
from transformers import SegformerFeatureExtractor

# dataset = load_dataset("imagefolder", data_dir="/content/lulc_png/")
feature_extractor = SegformerFeatureExtractor(reduce_labels=True)
train_dataset = SemanticSegmentationDataset(root_dir="/content/lulc_png/training", feature_extractor=feature_extractor)

['6560_3040.png', '8960_4320.png', '6240_4160.png', '4960_7680.png', '7360_4480.png', '5120_2720.png', '4480_5280.png', '7200_4000.png', '5280_8960.png', '6880_6080.png', '4160_7360.png', '6400_3840.png', '3520_7520.png', '7680_3040.png', '5440_3360.png', '7200_3040.png', '5440_7360.png', '3520_3680.png', '5280_8000.png', '3360_3680.png', '4640_7680.png', '8480_5280.png', '2240_6080.png', '5600_3840.png', '5440_5120.png', '6560_7200.png', '4000_4320.png', '3840_2240.png', '7040_5920.png', '6880_3360.png', '6720_7840.png', '3360_6080.png', '4160_1920.png', '7680_4320.png', '4320_5920.png', '4640_3840.png', '8000_6240.png', '2720_6240.png', '4960_6880.png', '5600_2400.png', '5920_5280.png', '7040_5760.png', '4320_4000.png', '6560_4480.png', '4960_3360.png', '4640_1280.png', '5920_4640.png', '4640_7360.png', '3360_7200.png', '3840_2080.png', '5440_4000.png', '7520_6240.png', '8960_4800.png', '5280_5440.png', '6560_2080.png', '5120_1120.png', '6400_4480.png', '6080_5120.png', '4000_7520.pn

/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [69]:
# dataset['train']['image']
id = [i for i in [0,31,63,95,127,159,191,223,255]]
string = [str(i) for i in range(9)]

In [70]:
id2color = {id: color for id, color in zip(id,string)}
label2id = {label: id for id, label in id2color.items()}
id2label = {id: label for id, label in id2color.items()}
label2id
id2label

{0: '0',
 31: '1',
 63: '2',
 95: '3',
 127: '4',
 159: '5',
 191: '6',
 223: '7',
 255: '8'}

In [71]:
from transformers import SegformerForSemanticSegmentation

# define model
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b0",
                                                         num_labels=9, 
                                                         id2label=id2label, 
                                                         label2id=label2id,
)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.2.proj.bias', 'decode_head.batch_norm.weight', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_n

In [43]:
from datasets import load_metric

metric = load_metric("mean_iou")

In [25]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 2770
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 190
    })
})

In [72]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [59]:
# enumerate(tqdm(train_dataloader))

In [ ]:
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

# define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(200):  # loop over the dataset multiple times
   print("Epoch:", epoch)
   for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs;
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits
        
        loss.backward()
        optimizer.step()

        # evaluate
        with torch.no_grad():
          upsampled_logits = nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
          predicted = upsampled_logits.argmax(dim=1)
          
          # note that the metric expects predictions + labels as numpy arrays
          metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())

        # let's print loss and metrics every 100 batches
        if idx % 100 == 0:
          # we use _compute for now which fixes an issue in speed
          # see this Github thread for more info: 
          metrics = metric.compute(num_labels = 9,ignore_index=True)

          print("Loss:", loss.item())
          print("Mean_iou:", metrics["mean_iou"])
          print("Mean accuracy:", metrics["mean_accuracy"])

Epoch: 0


  0%|          | 0/693 [00:00<?, ?it/s]

/root/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/927b58f57da3f4b6e385e47d8a4b3947ee3f7cfcdba9b9359eba2ada2ed6b951/mean_iou.py:257: RuntimeWarning: invalid value encountered in double_scalars
  all_acc = total_area_intersect.sum() / total_area_label.sum()
/root/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/927b58f57da3f4b6e385e47d8a4b3947ee3f7cfcdba9b9359eba2ada2ed6b951/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
/root/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/927b58f57da3f4b6e385e47d8a4b3947ee3f7cfcdba9b9359eba2ada2ed6b951/mean_iou.py:262: RuntimeWarning: Mean of empty slice
  metrics["mean_accuracy"] = np.nanmean(acc)


Loss: nan
Mean_iou: 0.0
Mean accuracy: nan
